In [1]:
GPT_CONFIG_124M = {
    "vocab_size":50257,
    "context_length":1024,
    "embedding_dim":768,
    "num_of_heads":12,
    "num_of_layers":12,
    "drop_rate":0.1,
    "qkv_bias":False
}

In [2]:
import torch
import torch.nn as nn

In [3]:
class FakeTransformer(nn.Module):
    def __init__(self,config):
        super().__init__()
        
    def forward(self,x):
        return x
    

In [4]:
class FakeLayerNormalization(nn.Module):
    def __init__(self,normalized_shape, eps=1e-5):
        super().__init__()
        
    def forward(self,x):
        return x
    

In [5]:
class GPTModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        
        self.token_embedding = nn.Embedding(config['vocab_size'], config['embedding_dim'])
        self.positional_embedding = nn.Embedding(config['context_length'], config['embedding_dim'])
        self.dropout_embedding = nn.Dropout(config['drop_rate'])
        
        self.transformer_blocks = nn.Sequential(
            *[FakeTransformer(config) for _ in range(config['num_of_layers'])]
        )
        
        self.final_normalization = FakeLayerNormalization(config['embedding_dim'])
        
        self.output_head = nn.Linear(config['embedding_dim'], config['vocab_size'], bias=False)
        
        
    def forward(self,inputs):
        batch_size, no_of_token = inputs.shape
        token_embedding = self.token_embedding(inputs)
        positional_embedding = self.positional_embedding(
            torch.arange(no_of_token, device=inputs.device)
        )
        
        x = token_embedding + positional_embedding
        x = self.dropout_embedding(x)
        x = self.transformer_blocks(x)
        x = self.final_normalization(x)
        logits = self.output_head(x)
        return logits
    
        

In [6]:
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')

batch = []

tx1 = "Every efforts moves you"
tx2 = "Everyday hold a"

batch.append(torch.tensor(tokenizer.encode(tx1)))
batch.append(torch.tensor(tokenizer.encode(tx2)))
batch = torch.stack(batch, dim=0)
batch

tensor([[6109, 4040, 6100,  345],
        [6109,  820, 1745,  257]])

In [7]:
batch.shape

torch.Size([2, 4])

In [8]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
logits = model(batch)
logits

tensor([[[-0.9289,  0.2748, -0.7557,  ..., -1.6070,  0.2702, -0.5888],
         [-0.8606,  1.3502, -0.2798,  ..., -0.6376,  1.4690,  1.2530],
         [ 0.5680,  1.6053, -0.2155,  ...,  1.1624,  0.1380,  0.7425],
         [ 0.0447,  2.4787, -0.8843,  ...,  1.3219, -0.0864, -0.5856]],

        [[-1.5474, -0.0542, -1.0571,  ..., -1.8061, -0.4494, -0.6747],
         [-0.9268, -0.0808, -0.4011,  ..., -0.3565,  0.3229,  0.9507],
         [-0.0063,  0.6248, -0.0776,  ...,  0.7499,  0.4245, -1.0798],
         [ 0.1666, -0.8138,  0.2307,  ...,  2.5035, -0.3055, -0.3083]]],
       grad_fn=<UnsafeViewBackward0>)

In [9]:
logits.shape

torch.Size([2, 4, 50257])